In [14]:
import pandas as pd
import numpy

core = pd.read_csv('raj-traders/core-data.csv')
bank = pd.read_csv('raj-traders/bank-statement.csv')
gst = pd.read_csv('raj-traders/gst.csv')
itr = pd.read_csv('raj-traders/itr.csv')
upi = pd.read_csv('raj-traders/upi.csv')
utility = pd.read_csv('raj-traders/utility.csv')

upi.head()

,Date,Direction,Amount (₹),Counterparty,UPI ID,Description
0,02-May-2025,IN,"₹23,400",Shree Hardware Co.,shreehardware@icici,Payment for Invoice #534
1,05-May-2025,OUT,"₹14,200",Power Utilities Punjab,punebillpay@upi,Electricity Bill – May
2,09-May-2025,OUT,"₹18,500",FastFix Tools Supplier,fastfix@axis,Raw material payment
3,12-May-2025,IN,"₹58,000",Premier Auto Ltd.,premierauto@hdfcbank,Order #PA721
4,16-May-2025,OUT,"₹7,600",Mobile Recharge,airtel.prepaid@upi,Employee phones


In [26]:
def calc_net_cashflow(df):
    # Convert Date to datetime for better parsing
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%Y')
    df['Month'] = df['Date'].dt.strftime('%b-%Y')
    
    # Clean amount column
    df['Amount (₹)'] = df['Amount (₹)'].apply(clean_amount)
    
    # Calculate monthly flows using groupby (much faster)
    monthly_summary = df.groupby(['Month', 'Direction'])['Amount (₹)'].sum().unstack(fill_value=0)
    
    # Calculate net cash flow
    monthly_summary['net_cash_flow'] = monthly_summary.get('IN', 0) - monthly_summary.get('OUT', 0)
    
    return monthly_summary.to_dict('index')

def volatility(summary):
    net_cashflows = [summary[i]['net_cash_flow'] for i in summary.keys()]
    stdev = numpy.std(net_cashflows)
    mean = sum(net_cashflows)/len(net_cashflows)
    return stdev/mean

def clean_amount(amount):
    if pd.isna(amount):
        return 0
    amount_str = str(amount).replace('₹', '').replace(',', '').strip()
    try:
        return int(float(amount_str)) if amount_str else 0
    except:
        return 0

def credit_summary(df):
    customers = set()
    summary = {}
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%Y')
    df['Month'] = df['Date'].dt.strftime('%b-%Y')
    df['Amount'] = df['Amount (₹)'].apply(clean_amount)
    for index, row in df.iterrows():
        month = row['Month']
        if month not in summary:
            summary[month] = {
                "new_customer" : 0,
                "amount_from_new" : 0,
                "repeat_customer" : 0,
                "amount_from_repeat" : 0
            }
        if row['Direction'] == 'IN':
            if row['UPI ID'] not in customers:
                summary[month]['new_customer'] += 1
                summary[month]['amount_from_new'] += row['Amount']
                customers.add(row['UPI ID'])
            else:
                summary[month]['repeat_customer'] += 1
                summary[month]['amount_from_repeat'] += row['Amount']
           
            
    return summary

def debit_summary(df):
    #PROBLEM - How to identify if debits are to vendors, should we compare net debit from all vendors or from previous known vendors
    vendors = set()
    summary = {}
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%Y')
    df['Month'] = df['Date'].dt.strftime('%b-%Y')
    df['Amount'] = df['Amount (₹)'].apply(clean_amount)
    for index, row in df.iterrows():
        month = row['Month']
        if month not in summary:
            net_debit = df.groupby(['Month', 'Direction'])['Amount (₹)'].sum().unstack(fill_value=0).get('IN', 0)
            summary[month] = {
                "new_debits" : 0,
                "amount_to_new" : 0
            }
        if row['Direction'] == 'OUT' and row['UPI ID'] not in vendors:
            summary[month]['new_debits'] += 1
            summary[month]['amount_to_new'] += row['Amount']
            vendors.add(row['UPI ID'])
    return summary
    
    
summary = calc_net_cashflow(upi)
summary
debit_summary(upi)
#volatility(summary)

        

{'May-2025': {'new_debits': 4, 'amount_to_new': 95300},
 nan: {'new_debits': 0, 'amount_to_new': 0},
 'Jun-2025': {'new_debits': 3, 'amount_to_new': 40250},
 'Jul-2025': {'new_debits': 3, 'amount_to_new': 33200}}